In [1]:
library("tidyverse")
library("data.table")
library("cowplot")

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.1
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘data.table’


The following objects are masked from ‘package:lubridate’:

    hour, isoweek, mday, minute, month, quarter, second, wday, week,
    yday, year


The following objects are masked from ‘package:dplyr’:

    between, first, last


The following object is masked from ‘package:purrr’:

    transpose



Attaching package: ‘cowplot’


The following object is masked from ‘package:lubridate’:

    stamp




In [2]:
invnorm <- function(x) {
  res <- rank(x)
  res <- qnorm(res/(length(res)+0.5))
  return(res)
}

In [3]:
extract_info <- function(the_var, sst, data, the_assay, the_state) {
    pval <- data[the_var, "Pr(>F)"]
    ss <- data["test_strain", "Sum Sq"]
    var_exp <- ss/sst
    ret <- data.table(assay = the_assay, var_name = the_var, hmm_state = the_state, pval = pval, var_exp = var_exp)
}

In [4]:
run_anova <- function(the_state, the_assay, data){
    fit <- lm(f_state_norm ~ date + time_numeric + tank_side + quadrant + test_strain, data = data[assay == the_assay & hmm_state == the_state])
    res <- aov(fit) |> summary()
    sst <- res[[1]][, "Sum Sq"] |> sum()
    ret <- lapply(
        rownames(res[[1]]) |> trimws(), 
        extract_info,
        sst = sst,
        data = res[[1]],
        the_assay = the_assay,
        the_state = the_state
    ) |> rbindlist()
    return(ret)
}

In [5]:
df <- fread("/nfs/research/birney/users/saul/nextflow/medaka_behaviour_pilot/hmm/time_step0.08_n_states15_hmm.csv.gz")
df[, mean_dist := log10(mean(distance)), by = hmm_state]
tmp <- df[, .(hmm_state, mean_dist)] |> distinct() |> as.data.table()
tmp[, hmm_state_recoded := rank(mean_dist)]
df <- merge(df, tmp, by = c("hmm_state", "mean_dist"))
head(df)

hmm_state,mean_dist,id,frame_n,time_s,distance,angle,hmm_state_recoded
<int>,<dbl>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
0,0.1005448,20190611_1331_icab_icab_R_no_q1_ref,757,25.23333,0.2083191,0.0000000,6
0,0.1005448,20190611_1331_icab_icab_R_no_q1_ref,761,25.36667,0.4425455,0.0000000,6
0,0.1005448,20190611_1331_icab_icab_R_no_q1_ref,1755,58.50000,1.2756671,-0.4597774,6
0,0.1005448,20190611_1331_icab_icab_R_no_q1_ref,1779,59.30000,1.3794776,-0.1900450,6
0,0.1005448,20190611_1331_icab_icab_R_no_q1_ref,1781,59.36667,1.3745458,-0.3463498,6
0,0.1005448,20190611_1331_icab_icab_R_no_q1_ref,1787,59.56667,1.4936201,-0.3414281,6


In [6]:
df_sum <- df[, .(id, hmm_state = hmm_state_recoded)] |>
    separate(
        id,
        into = c("date", "time_string", "ref_strain", "test_strain", "tank_side", "assay", "quadrant", "fish_type"),
        sep = "_",
        remove = FALSE
    ) |>
    as.data.table()

df_sum[, time_numeric_h := str_remove(time_string, "..$") |> as.numeric()]
df_sum[, time_numeric_m := str_remove(time_string, "^..") |> as.numeric()]
df_sum[, time_numeric := time_numeric_h + (time_numeric_m/60)]

head(df_sum)

id,date,time_string,ref_strain,test_strain,tank_side,assay,quadrant,fish_type,hmm_state,time_numeric_h,time_numeric_m,time_numeric
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>
20190611_1331_icab_icab_R_no_q1_ref,20190611,1331,icab,icab,R,no,q1,ref,6,13,31,13.51667
20190611_1331_icab_icab_R_no_q1_ref,20190611,1331,icab,icab,R,no,q1,ref,6,13,31,13.51667
20190611_1331_icab_icab_R_no_q1_ref,20190611,1331,icab,icab,R,no,q1,ref,6,13,31,13.51667
20190611_1331_icab_icab_R_no_q1_ref,20190611,1331,icab,icab,R,no,q1,ref,6,13,31,13.51667
20190611_1331_icab_icab_R_no_q1_ref,20190611,1331,icab,icab,R,no,q1,ref,6,13,31,13.51667
20190611_1331_icab_icab_R_no_q1_ref,20190611,1331,icab,icab,R,no,q1,ref,6,13,31,13.51667


In [7]:
df_dge <- df_sum[
    fish_type == "test" | test_strain == "icab", # I can use both fish in icab/icab pairs
    .(hmm_state, n_tot = .N), by = c("id", "date", "time_numeric", "test_strain", "tank_side", "assay", "quadrant", "fish_type")
][
    , .(f_state = .N/unique(n_tot)), by = c("hmm_state", "id", "date", "time_numeric", "test_strain", "tank_side", "assay", "quadrant", "fish_type")
]
head(df_dge)

hmm_state,id,date,time_numeric,test_strain,tank_side,assay,quadrant,fish_type,f_state
<dbl>,<chr>,<chr>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
6,20190611_1331_icab_icab_R_no_q1_ref,20190611,13.51667,icab,R,no,q1,ref,0.06634808
10,20190611_1331_icab_icab_R_no_q1_ref,20190611,13.51667,icab,R,no,q1,ref,0.02256057
11,20190611_1331_icab_icab_R_no_q1_ref,20190611,13.51667,icab,R,no,q1,ref,0.06868193
3,20190611_1331_icab_icab_R_no_q1_ref,20190611,13.51667,icab,R,no,q1,ref,0.06490331
9,20190611_1331_icab_icab_R_no_q1_ref,20190611,13.51667,icab,R,no,q1,ref,0.17470549
2,20190611_1331_icab_icab_R_no_q1_ref,20190611,13.51667,icab,R,no,q1,ref,0.06645921


In [8]:
df_dge[, .N, by = test_strain]

test_strain,N
<chr>,<int>
icab,4043
kaga,1389
hni,2232
hdr,1723
ho5,1652


In [9]:
df_dge[, .N, by = fish_type]

fish_type,N
<chr>,<int>
ref,2017
test,9022


In [10]:
df_dge[, .(res = sum(f_state) - 1 < .Machine$double.eps), by = id][, all(res)] |> stopifnot()

In [11]:
df_dge[, f_state_norm := invnorm(f_state), by = c("assay", "hmm_state")]

In [12]:
res <- list(
    of = lapply(1:15, run_anova, data = df_dge, the_assay = "of"),
    no = lapply(1:15, run_anova, data = df_dge, the_assay = "no")
) |>
    lapply(rbindlist) |>
    rbindlist()

In [13]:
head(res)

assay,var_name,hmm_state,pval,var_exp
<chr>,<chr>,<int>,<dbl>,<dbl>
of,date,1,3.448927e-01,0.2974121
of,time_numeric,1,4.041694e-01,0.2974121
of,tank_side,1,9.265892e-01,0.2974121
of,quadrant,1,3.202357e-01,0.2974121
of,test_strain,1,7.831160e-27,0.2974121
of,Residuals,1,NA,0.2974121


In [14]:
res <- res[var_name != "Residuals"]

In [15]:
res[, q := p.adjust(pval, method = "fdr")]
res[, sig := q < 0.05]

In [16]:
res[var_name == "test_strain", .(n = .N, min = min(q) |> signif(3), max = max(q) |> signif(3)), by = c("sig", "assay")]

sig,assay,n,min,max
<lgl>,<chr>,<int>,<dbl>,<dbl>
TRUE,of,11,1.17e-24,0.023600
FALSE,of,4,1.98e-01,0.380000
TRUE,no,8,2.79e-13,0.000156
FALSE,no,7,2.93e-01,0.868000


In [17]:
res[var_name == "test_strain", max(var_exp) |> signif(3)]

[1] 0.297

In [18]:
res[var_name != "test_strain" & sig == TRUE, .(max = max(q), min = min(q)), by = c("assay", "var_name")]

assay,var_name,max,min
<chr>,<chr>,<dbl>,<dbl>
of,date,0.03938738,2.805763e-07
no,quadrant,0.03221769,1.812955e-02
no,date,0.04457375,4.417953e-08


In [19]:
res[var_name %in% c("quadrant", "date"), .(max = max(q) |> signif(3), min = min(q) |> signif(3)), by = c("assay")]

assay,max,min
<chr>,<dbl>,<dbl>
of,0.899,2.81e-07
no,0.968,4.42e-08


In [22]:
pretty_df <- res[
    var_name == "test_strain",
    .(
        `Assay component` = ifelse(assay == "of", "Open field", "Novel object"),
        `HMM State` = hmm_state,
        `p-value (FDR-adjusted)` = signif(q, 3) |> format(scientific = TRUE),
        `Variance explained` = signif(var_exp, 3)
    )
]

pretty_df

Assay component,HMM State,p-value (FDR-adjusted),Variance explained
<chr>,<int>,<chr>,<dbl>
Open field,1,1.17e-24,0.29700
Open field,2,4.75e-07,0.10100
Open field,3,6.42e-07,0.09770
Open field,4,6.06e-21,0.26100
Open field,5,2.36e-02,0.03580
Open field,6,1.73e-02,0.03840
Open field,7,5.54e-07,0.09640
Open field,8,3.76e-05,0.06990
Open field,9,8.54e-05,0.06730


In [23]:
fwrite(pretty_df, "tableS1.csv")